In [217]:
import pandas as pd
pd.options.mode.chained_assignment = None

import re
import nltk
import contractions

tokenizer = nltk.RegexpTokenizer(r"\w+")
lemmatizer = nltk.WordNetLemmatizer()

SUBREDDIT = 'ukraine'
filename = f'./data/{SUBREDDIT}_raw_pushshift_data.csv'
dataframe = pd.read_csv(filename)
known_word_list = ['media', 'ss', 'gates']

C:\Users\1408\AppData\Local\Temp\ipykernel_17336\2184716311.py:13: DtypeWarning: Columns (86,87,88) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(filename)


preprocessing with
* lowercase
* remove urls, usertags
* join contractions
* remove punctuation
* tokenize
* lemmatize

In [218]:
def preprocessing(submission):
    submission = str(submission)

    submission = " ".join(submission.split())

    submission = submission.lower()
    submission = re.sub(r"^(\d{1,4})([.])(\d{1,2})(([.])(\d{1,4}))?$", r"\1 \3 \6", submission)

    #source https://www.regextester.com/93652
    submission = re.sub(r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/[^ ]*)?","",submission)
    
    #source https://www.regextester.com/94502  
    #submission = re.sub(r"(?:http(s)?:\/\/)?[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&'\(\)\*\+,;=.%]+","",submission)
    submission = re.sub(r"(@\[A-Za-z0-9]+)|&gt;|&amp;|#x200b;","",submission)

    submission = re.sub("[/:\-;*!?]+", " ", submission)
    submission = re.sub("[.]{2,}", " ", submission)

    submission = re.sub(r"[^0-9A-Za-z' ]","",submission)
    submission = " ".join(contractions.fix(word) for word in submission.split())
    submission = re.sub(r"'s|’s|'","",submission)
    submission = tokenizer.tokenize(submission)
    lemmatized_submission = []   
    for word in submission:
        pos = get_pos_tag([word])
        if pos == None or word in known_word_list:
            lemmatized_submission.append(word)
        else:
            lemmatized_submission.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_submission

In [219]:
def get_pos_tag(word):
    pos = nltk.pos_tag(word)
    if pos[0][1].startswith('J'):
        return 'a'
    elif pos[0][1].startswith('V'):
        return 'v'
    elif pos[0][1].startswith('R'):
        return 'r'
    elif pos[0][1].startswith('N'):
        return 'n'
    else:
        return None

apply preprocessing on title and selftext

In [220]:
dataframe['title'] = dataframe['title'].apply(preprocessing)
dataframe['selftext'] = dataframe['selftext'].apply(preprocessing)

In [221]:
wordlist = ["ukraine", "ukrainian", "ukraini", "russia", "russian", "putin", "zelensky", "zelenskyy", "nato", "kiev", "kyiv", "sanction", "oil", "gas", "swift", "moscow", "kremlin", "azov", "vladimir", "war"]

def checkForWords(array):
    for word in wordlist:
        if word in array:
            return True
    return False

filter only for submissions where either title or self text contains certain ukraine war related words

In [222]:
print(dataframe.shape)
filtered_dataframe = dataframe.loc[dataframe["title"].apply(checkForWords) | dataframe["selftext"].apply(checkForWords)]
filtered_dataframe['title'] = filtered_dataframe['title'].apply(" ".join)
filtered_dataframe['selftext'] = filtered_dataframe['selftext'].apply(" ".join)
filtered_dataframe.to_csv(f'./data/{SUBREDDIT}_preprocessed_filtered.csv', header=True, index=False, columns=list(filtered_dataframe.axes[1]), encoding='utf-8')
filtered_dataframe.shape

(36737, 89)


(25590, 89)